In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.metrics import recall_score, accuracy_score, confusion_matrix
import plotly.express as px
import warnings

In [3]:
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Inputting Data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [ ]:
train_df.info()

In [ ]:
print(train_df.isnull().sum())
test_df.isnull().sum()

In [6]:
train_df.drop(['id'], axis=1, inplace=True)

In [7]:
def colchange(train_df):
    train_df['Sex'] = train_df['Sex'].replace(['F', 'M'], [0, 1])
    train_df['Sex'] = pd.to_numeric(train_df['Sex'])

    train_df['Ascites'] = train_df['Ascites'].replace(['N', 'Y'], [0, 1])
    train_df['Ascites'] = pd.to_numeric(train_df['Ascites'])

    train_df['Hepatomegaly'] = train_df['Hepatomegaly'].replace(['N', 'Y'], [0, 1])
    train_df['Hepatomegaly'] = pd.to_numeric(train_df['Hepatomegaly'])

    train_df['Spiders'] = train_df['Spiders'].replace(['N', 'Y'], [0, 1])
    train_df['Spiders'] = pd.to_numeric(train_df['Spiders'])

    train_df['Edema'] = train_df['Edema'].replace(['N', 'S', 'Y'], [0.2, 0.4, 0.6])
    train_df['Edema'] = pd.to_numeric(train_df['Edema'])

    train_df['Drug'] = train_df['Drug'].replace(['D-penicillamine', 'Placebo'], [0, 1])
    train_df['Drug'] = pd.to_numeric(train_df['Drug'])
    return train_df

train_df = colchange(train_df)
test_df = colchange(test_df)

train_df['Status'] = train_df['Status'].replace(['C', 'CL', 'D'], [0, 1, 2])
train_df['Status'] = pd.to_numeric(train_df['Status'])

In [8]:
def scale(X):
    from sklearn import preprocessing
    scaled=preprocessing.StandardScaler()
    scaler=scaled.fit(X)
    X=scaler.transform(X)
    return X

In [9]:
def evaluate(y_test,ypred):
    from sklearn.metrics import recall_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    import plotly.express as px
    y_test['Status']=y_test['Status'].replace(['C', 'CL', 'D'],[1,2,3])
    y_pred=pd.DataFrame(data=ypred, columns=['Status'])
    y_pred['Status']=y_pred['Status'].replace(['C', 'CL', 'D'],[1,2,3])

    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(20, 20))
    sns.heatmap(cm,annot=True, cmap='Blues')

In [ ]:
train_df['Status']=pd.Categorical(train_df['Status'])
X=train_df[['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders',
       'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos',
       'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']]
y=train_df[['Status']]
train_df['Status']=train_df['Status'].replace(['C', 'CL', 'D'],[0.2,0.4,0.6])
corr = train_df.select_dtypes('number').corr()
plt.figure(figsize=(20, 20))
s=sns.heatmap(corr,annot=True, cmap='Blues')

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(scale(X),y,train_size=0.8, random_state=42)
y_train.info()

In [12]:
def scale(X):
    scaled = preprocessing.StandardScaler()
    scaler = scaled.fit(X)
    X = scaler.transform(X)
    return X


In [13]:
# Feature Selection
X = train_df[['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders',
              'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos',
              'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']]

y = train_df['Status']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(scale(X), y, train_size=0.7, random_state=42)

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
print(nb.score(X_train, y_train))
y_pred = nb.predict(X_test)


In [ ]:
def evaluate(y_test, y_pred):
    print("Accuracy: ", accuracy_score(y_test, y_pred))
    print("Recall Score: ", recall_score(y_test, y_pred, average='macro'))
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, cmap='Blues')
evaluate(y_test, y_pred)

In [17]:
X_test = test_df.drop(['id'], axis=1)
y_pred_proba = nb.predict_proba(scale(X_test))
assert y_pred_proba.shape == (test_df.shape[0], 3)
submission_labels = ["Status_C", "Status_CL", "Status_D"]
submission = pd.DataFrame({"id": test_df.id, **dict(zip(submission_labels, y_pred_proba.T))})
submission.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
submission = pd.read_csv('submission.csv')
threshold = 0.5
submission['Survival'] = submission['Status_C'] > threshold
for index, row in submission.iterrows():
    print(f"Patient {index}: {'Survive' if row['Survival'] else 'Not Survive'}") 